In [18]:
import pandas as pd

In [19]:
df = pd.read_csv('train.csv')
df

,ID,frag_1,frag_2,h_bond_distance
0,TRAIN_0000000,TLGR,VALV,15.0
1,TRAIN_0000001,AILTCPF,GRIVPRF,20.0
2,TRAIN_0000002,LRLSCA,MQLYVT,17.0
3,TRAIN_0000003,TASVVCLLNN,SLTLTSSLSY,25.0
4,TRAIN_0000004,VKLYL,ILVSD,19.0
...,...,...,...,...
1699062,TRAIN_1699062,LMLVHYEGYL,IELLDINFIL,24.0
1699063,TRAIN_1699063,MLIT,IFPV,16.0
1699064,TRAIN_1699064,IYLGRYEGWYS,MYNEESVWTVV,24.0
1699065,TRAIN_1699065,KFWLIYTD,CTTLYNHC,21.0


Dataset Info.

train.csv [파일]

ID : 부여번호

frag_1 : Protein 조각 sequence 1

frag_2 : Protein 조각 sequence 2

h_bond_distance : Hydrogen bond distance (Target)

------------------------
test.csv [파일]

ID : 부여번호

frag_1 : Protein 조각 sequence 1

frag_2 : Protein 조각 sequence 2

-----------------------
sample_submission.csv [제출양식]

ID : 부여번호

h_bond_distance : 예측한 Hydrogen bond distance (Target)


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1699067 entries, 0 to 1699066
Data columns (total 4 columns):
 #   Column           Dtype  
---  ------           -----  
 0   ID               object 
 1   frag_1           object 
 2   frag_2           object 
 3   h_bond_distance  float64
dtypes: float64(1), object(3)
memory usage: 51.9+ MB


In [21]:
df.describe()

,h_bond_distance
count,1.699067e+06
mean,1.929865e+01
std,3.972532e+00
min,4.000000e+00
25%,1.700000e+01
50%,1.900000e+01
75%,2.100000e+01
max,3.700000e+01


In [22]:
df['frag_1'].value_counts()

frag_1
FSGSGS        3333
TAALGCLVKD    3140
TASVVCLLNN    2936
FTISRD        2639
SQESV         2625
              ... 
SATALV           1
TVSFLQP          1
VHAFPALL         1
GVMGVQIR         1
IEVVSM           1
Name: count, Length: 255078, dtype: int64

In [23]:
df['frag_2'].value_counts()

frag_2
ITLTFD         3884
PVTVVSSLSY     3194
SLTLTSSLSY     2978
SSLSY          2586
KKDVKT         2573
               ... 
DFEIWGMVGRA       1
RVPQ              1
KVAC              1
LFNNWE            1
VYFKVT            1
Name: count, Length: 260058, dtype: int64

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_log_error
import numpy as np

# 올바르게 fit된 vectorizer 사용
frag_all_train = df['frag_1'] + " " + df['frag_2']

# 벡터화
tfidf = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))
X_df = tfidf.fit_transform(frag_all_train)
y = df['h_bond_distance']

# 학습/검증 분리
X_train, X_val, y_train, y_val = train_test_split(X_df, y, test_size=0.2, random_state=42)

# 모델 학습
model = LGBMRegressor()
model.fit(X_train, y_train)

# 평가
preds = model.predict(X_val)
rmsle = np.sqrt(mean_squared_log_error(y_val, preds))
print(f"RMSLE: {rmsle:.4f}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 19.926187 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 661693
[LightGBM] [Info] Number of data points in the train set: 1359253, number of used features: 9631
[LightGBM] [Info] Start training from score 19.297520
RMSLE: 0.1089


In [25]:
tdf = pd.read_csv('test.csv')
tdf

,ID,frag_1,frag_2
0,TEST_000000,VVLASA,FKVVNV
1,TEST_000001,FNLR,GITY
2,TEST_000002,IIMD,WHLD
3,TEST_000003,DFSE,QMEA
4,TEST_000004,VLLR,AVLA
...,...,...,...
163532,TEST_163532,GMVRCDTA,GYYGDGGQ
163533,TEST_163533,EMTFFSVKNIKKKLP,TIKDKVKMPIRKILF
163534,TEST_163534,SQESV,SSLSY
163535,TEST_163535,GYIH,YDIK


In [26]:
# 올바르게 fit된 vectorizer 사용
frag_all_test = tdf['frag_1'] + " " + tdf['frag_2']

# 벡터화
X_whatiwant = tfidf.transform(frag_all_test)

In [27]:
preds = model.predict(X_whatiwant)


submission = pd.DataFrame({
    'ID': tdf['ID'],
    'h_bond_distance': preds
})
submission.to_csv('submission.csv', index=False)
print("🌞Complete")

🌞Complete
